In [ ]:
#importing libraries
import threading
import socket
import time

In [ ]:
#initialising computer's socket to communicate with tello
#using port 9000
loc_addr = ("", 9000)

sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
sock.bind(loc_addr)

In [ ]:
#tello's address on it's own LAN
#port 8889 to send commands
#port 8890 to get tello state
#port 11111 to get video stream if video stream is set to on (covered later)
tello_addr = ("192.168.10.1", 8889)

In [ ]:
#receive function to get reply message from tello (error/ack message)
def receive():
    #128 represents max number of bytes in reply message
    data, server = sock.recvfrom(128)
    #print decoded reply from tello
    print (data.decode(encoding="utf-8"))

In [ ]:
#set tello to command mode
sock.sendto("command".encode(encoding="utf-8"), tello_addr)
#get ack/error message from tello
receive()

In [ ]:
#basic commands

#takeoff
sock.sendto("takeoff".encode(encoding="utf-8"), tello_addr)
receive()
#move up 50cm
sock.sendto("up 50".encode(encoding="utf-8"), tello_addr)
receive()
#move right 50cm
sock.sendto("right 50".encode(encoding="utf-8"), tello_addr)
receive()
#move down 50cm
sock.sendto("down 50".encode(encoding="utf-8"), tello_addr)
receive()
#move left 50cm
sock.sendto("left 50".encode(encoding="utf-8"), tello_addr)
receive()
#turn clockwise 90 degrees
sock.sendto("cw 90".encode(encoding="utf-8"), tello_addr)
receive()
#move forward 50cm
sock.sendto("forward 50".encode(encoding="utf-8"), tello_addr)
receive()
#move backwards 50cm
sock.sendto("back 50".encode(encoding="utf-8"), tello_addr)
receive()
#turn counter-clockwise 90 degrees
sock.sendto("ccw 90".encode(encoding="utf-8"), tello_addr)
receive()
#land
sock.sendto("land".encode(encoding="utf-8"), tello_addr)
receive()


In [ ]:
#get video feed using tellopy library
from tellopy import Tello
import av
import cv2
import numpy as np
import imutils

tello = Tello()
tello.set_loglevel(0)
tello.connect()
tello.wait_for_connection(3)



In [ ]:
# Turn on video feed of drone and access it
tello.start_video()
retry = 3
container = None
while container is None and retry > 0:
    retry -= 1
    try:
        container = av.open(tello.get_video_stream())
        container.streams.video[0].thread_type = 'AUTO'
    except av.AVError as ave:
        print(ave)
        print('retry...')

In [ ]:
running = True
while running:
    #skip initial frames
    frame_skip = 300
    for frame in container.decode(video=0):
        if frame_skip > 0:
            frame_skip -= 1
            continue
        # Get an image from frame
        image = cv2.cvtColor(np.array(frame.to_image()), cv2.COLOR_RGB2BGR)
        # Resize the image
        image = imutils.resize(image, width=800)
        # Display current frame on screen
        cv2.imshow("image", image)
        key = cv2.waitKey(1) & 0xFF

        #terminate when q is pressed
        if key == ord("q"):
            running = False
            break
        #skip 5 frames to prevent lag
        frame_skip = 7